In [1]:
# %load_ext autoreload
# %autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import os
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN, eval_nn, train_mlp, test_mlp, train_mlpbeta, test_mlpbeta, train_cnn, test_cnn

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch
set_gpu()

Num GPUs Available:  1


In [2]:
sub_type = 'AB'
with open('train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,feat,feat_sq = pickle.load(f)

In [3]:
## Training 
sub = 2
train_grp = 2
n_train = 'fullallmix4'
train_scale = 5
cv_type = 'manual'
scaler_load = False
feat_type = 'mav'

ind = (params[:,0] == sub) & (params[:,3] == train_grp)

x_train, x_test, x_valid, p_train, p_test, p_valid = prd.train_data_split(raw,params,sub,sub_type,dt=cv_type,load=True,train_grp=train_grp)

emg_scale = np.ones((np.size(x_train,1),1))
for i in range(np.size(x_train,1)):
    emg_scale[i] = 5/np.max(np.abs(x_train[:,i,:]))
x_train = x_train*emg_scale
x_valid = x_valid*emg_scale

x_train_noise, x_train_clean, y_train_clean = prd.add_noise(x_train, p_train, sub, n_train, train_scale)
x_valid_noise, x_valid_clean, y_valid_clean = prd.add_noise(x_valid, p_valid, sub, n_train, train_scale)

# shuffle data to make even batches
x_train_noise, x_train_clean, y_train_clean = shuffle(x_train_noise, x_train_clean, y_train_clean, random_state = 0)

# Extract features
scaler = MinMaxScaler(feature_range=(0,1))
x_train_noise_cnn, scaler = prd.extract_scale(x_train_noise,scaler,scaler_load,ft=feat_type,emg_scale=emg_scale) 
x_valid_noise_cnn, _ = prd.extract_scale(x_valid_noise,scaler,ft=feat_type,emg_scale=emg_scale)
x_train_noise_cnn = x_train_noise_cnn.astype('float32')
x_valid_noise_cnn = x_valid_noise_cnn.astype('float32')

# reshape data for nonconvolutional network
x_train_noise_mlp = x_train_noise_cnn.reshape(x_train_noise_cnn.shape[0],-1)
x_valid_noise_mlp = x_valid_noise_cnn.reshape(x_valid_noise_cnn.shape[0],-1)

# create batches
trainmlp_ds = tf.data.Dataset.from_tensor_slices((x_train_noise_mlp, y_train_clean)).batch(128)
testmlp_ds = tf.data.Dataset.from_tensor_slices((x_valid_noise_mlp, y_valid_clean)).batch(128)
traincnn_ds = tf.data.Dataset.from_tensor_slices((x_train_noise_cnn, y_train_clean)).batch(128)
testcnn_ds = tf.data.Dataset.from_tensor_slices((x_valid_noise_cnn, y_valid_clean)).batch(128)

Loading training data: traindata_manual/AB2_traindata_2.p


In [8]:
mlp = MLP()
mlp_beta = MLPbeta()
cnn = CNN()

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [9]:
## TRAIN TEST MLP
@tf.function
def train_mlp(x, y):
    with tf.GradientTape() as tape:
        y_out = mlp(x)
        loss = loss_fn(y, y_out)
    gradients = tape.gradient(loss, mlp.trainable_variables)
    optimizer.apply_gradients(zip(gradients, mlp.trainable_variables))

    train_loss(loss)
    train_accuracy(y, y_out)

@tf.function
def test_mlp(x, y):
    y_out = mlp(x)
    t_loss = loss_fn(y, y_out)

    test_loss(t_loss)
    test_accuracy(y, y_out)

## TRAIN TEST MLP BETA
@tf.function
def train_mlpbeta(x, y):
    with tf.GradientTape() as tape:
        y_out = mlp_beta(x)
        loss = loss_fn(y, y_out)
    gradients = tape.gradient(loss, mlp_beta.trainable_variables)
    optimizer.apply_gradients(zip(gradients, mlp_beta.trainable_variables))

    train_loss(loss)
    train_accuracy(y, y_out)

@tf.function
def test_mlpbeta(x, y):
    y_out = mlp_beta(x)
    t_loss = loss_fn(y, y_out)

    test_loss(t_loss)
    test_accuracy(y, y_out)

## TRAIN TEST CNN
@tf.function
def train_cnn(x, y):
    with tf.GradientTape() as tape:
        y_out = cnn(x)
        loss = loss_fn(y, y_out)
    gradients = tape.gradient(loss, cnn.trainable_variables)
    optimizer.apply_gradients(zip(gradients, cnn.trainable_variables))

    train_loss(loss)
    train_accuracy(y, y_out)

@tf.function
def test_cnn(x, y):
    y_out = cnn(x)
    t_loss = loss_fn(y, y_out)

    test_loss(t_loss)
    test_accuracy(y, y_out)

In [10]:
EPOCHS = 60
models = ['mlp','mlpbeta','cnn']

for model in models:
    print('Training ' + model)
    for epoch in range(EPOCHS):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        # Train MLP
        if model == 'mlp':
            for x, y in trainmlp_ds:
                train_mlp(x, y)
            for x_test, y_test in testmlp_ds:
                test_mlp(x_test, y_test)
        # Train MLP Beta
        elif model == 'mlpbeta':
            for x, y in trainmlp_ds:
                train_mlpbeta(x, y)
            for x_test, y_test in testmlp_ds:
                test_mlpbeta(x_test, y_test)
        # Train CNN
        elif model == 'cnn':
            for x, y in traincnn_ds:
                train_cnn(x,y)
            for x_test, y_test in testcnn_ds:
                test_cnn(x_test, y_test)

        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}'
        )

Training mlp
Epoch 1, Loss: 1.9436055421829224, Accuracy: 15.567461013793945, Test Loss: 1.9376263618469238, Test Accuracy: 17.873016357421875
Epoch 2, Loss: 1.9159562587738037, Accuracy: 21.91269874572754, Test Loss: 1.881170392036438, Test Accuracy: 25.587303161621094
Epoch 3, Loss: 1.845521092414856, Accuracy: 29.738096237182617, Test Loss: 1.814572811126709, Test Accuracy: 34.0476188659668
Epoch 4, Loss: 1.796665906906128, Accuracy: 35.4603157043457, Test Loss: 1.7839698791503906, Test Accuracy: 36.777774810791016
Epoch 5, Loss: 1.7720528841018677, Accuracy: 38.4523811340332, Test Loss: 1.7745262384414673, Test Accuracy: 36.5396842956543
Epoch 6, Loss: 1.7582334280014038, Accuracy: 39.81349182128906, Test Loss: 1.7709163427352905, Test Accuracy: 38.20634841918945
Epoch 7, Loss: 1.7474658489227295, Accuracy: 41.32539749145508, Test Loss: 1.7668397426605225, Test Accuracy: 37.73015594482422
Epoch 8, Loss: 1.7361502647399902, Accuracy: 42.31349182128906, Test Loss: 1.7665700912475586,

In [5]:
EPOCHS = 60
models = ['mlp','mlpbeta','cnn']

for model in models:
    print('Training ' + model)
    for epoch in range(EPOCHS):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        # Train MLP
        if model == 'mlp':
            for x, y in trainmlp_ds:
                train_mlp(x, y, mlp, loss_fn, optimizer, train_loss, train_accuracy)
            for x_test, y_test in testmlp_ds:
                test_mlp(x_test, y_test, mlp, loss_fn, test_loss, test_accuracy)
        # Train MLP Beta
        elif model == 'mlpbeta':
            for x, y in trainmlp_ds:
                train_mlpbeta(x, y, mlp_beta, loss_fn, optimizer, train_loss, train_accuracy)
            for x_test, y_test in testmlp_ds:
                test_mlpbeta(x_test, y_test, mlp_beta, loss_fn, test_loss, test_accuracy)
        # Train CNN
        elif model == 'cnn':
            for x, y in traincnn_ds:
                train_cnn(x, y, cnn, loss_fn, optimizer, train_loss, train_accuracy)
            for x_test, y_test in testcnn_ds:
                test_cnn(x_test, y_test, cnn, loss_fn, test_loss, test_accuracy)

        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}'
        )

Training mlp
Epoch 1, Loss: 1.9400993585586548, Accuracy: 16.297618865966797, Test Loss: 1.919284701347351, Test Accuracy: 18.174604415893555
Epoch 2, Loss: 1.8843544721603394, Accuracy: 25.361112594604492, Test Loss: 1.8650637865066528, Test Accuracy: 27.841270446777344
Epoch 3, Loss: 1.8406009674072266, Accuracy: 32.019840240478516, Test Loss: 1.822227954864502, Test Accuracy: 36.22222137451172
Epoch 4, Loss: 1.7912793159484863, Accuracy: 38.2023811340332, Test Loss: 1.7810697555541992, Test Accuracy: 38.69841384887695
Epoch 5, Loss: 1.755397915840149, Accuracy: 41.32143020629883, Test Loss: 1.7560169696807861, Test Accuracy: 40.71428298950195
Epoch 6, Loss: 1.7384065389633179, Accuracy: 42.5396842956543, Test Loss: 1.7377867698669434, Test Accuracy: 42.14285659790039
Epoch 7, Loss: 1.7273625135421753, Accuracy: 43.626983642578125, Test Loss: 1.728888988494873, Test Accuracy: 43.22222137451172
Epoch 8, Loss: 1.7178070545196533, Accuracy: 44.69444274902344, Test Loss: 1.72321593761444

In [11]:
mlp_enc = mlp.get_layer(name='enc')
mlpbeta_enc = mlp_beta.get_layer(name='enc')
cnn_enc = cnn.get_layer(name='enc')

mlp_aligned = mlp_enc(x_train_noise_mlp).numpy()
mlp_beta_aligned = mlpbeta_enc(x_train_noise_mlp).numpy()
cnn_aligned = cnn_enc(x_train_noise_cnn).numpy()

y_train_aligned = np.argmax(y_train_clean, axis=1)[...,np.newaxis]
w_mlp, c_mlp,_, _, _ = train_lda(mlp_aligned,y_train_aligned)
w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlp_beta_aligned,y_train_aligned)
w_cnn, c_cnn,_, _, _ = train_lda(cnn_aligned,y_train_aligned)

In [12]:
test_grp = 4
cv_type = 'manual'
n_test = 'partposrealmixeven24'

with open('real_noise/all_real_noise.p', 'rb') as f:
    real_noise_temp, _ = pickle.load(f)

_, x_test, _, _, p_test, _ = prd.train_data_split(raw,params,sub,sub_type,dt=cv_type,train_grp=test_grp)
clean_size = int(np.size(x_test,axis=0))
x_test = x_test*emg_scale

x_test_noise, x_test_clean, y_test_clean = prd.add_noise(x_test, p_test, sub, n_test, 1, real_noise=real_noise_temp, emg_scale = emg_scale)
x_test_cnn, _ = prd.extract_scale(x_test_noise,scaler,ft=feat_type,emg_scale=emg_scale)
x_test_cnn = x_test_cnn.astype('float32')
x_test_mlp = x_test_cnn.reshape(x_test_cnn.shape[0],-1)

mlp_test_aligned = mlp_enc(x_test_mlp).numpy()
mlpbeta_test_aligned = mlpbeta_enc(x_test_mlp).numpy()
cnn_test_aligned = cnn_enc(x_test_cnn).numpy()
y_test_aligned = np.argmax(y_test_clean, axis=1)[...,np.newaxis]

clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test_clean,mlp,clean_size)
clean_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
noisy_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
print(
    f'MLP ---- '
    f'Clean Accuracy: {clean_acc * 100:.2f}, '
    f'Noisy Accuracy: {noisy_acc * 100:.2f}, '
    f'LDA Clean Accuracy: {clean_lda * 100:.2f}, '
    f'LDA Noisy Accuracy: {noisy_lda * 100:.2f}'
)

clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test_clean,mlp_beta,clean_size)
clean_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
noisy_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
print(
    f'MLPB ---- '
    f'Clean Accuracy: {clean_acc * 100:.2f}, '
    f'Noisy Accuracy: {noisy_acc * 100:.2f}, '
    f'LDA Clean Accuracy: {clean_lda * 100:.2f}, '
    f'LDA Noisy Accuracy: {noisy_lda * 100:.2f}'
)

clean_acc, noisy_acc = eval_nn(x_test_cnn, y_test_clean,cnn,clean_size)
clean_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
noisy_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
print(
    f'CNN ---- '
    f'Clean Accuracy: {clean_acc * 100:.2f}, '
    f'Noisy Accuracy: {noisy_acc * 100:.2f}, '
    f'LDA Clean Accuracy: {clean_lda * 100:.2f}, '
    f'LDA Noisy Accuracy: {noisy_lda * 100:.2f}'
)

Loading training data: traindata_manual/AB2_traindata_4.p
MLP ---- Clean Accuracy: 76.31, Noisy Accuracy: 61.20, LDA Clean Accuracy: 73.74, LDA Noisy Accuracy: 60.20
MLPB ---- Clean Accuracy: 84.40, Noisy Accuracy: 62.06, LDA Clean Accuracy: 86.57, LDA Noisy Accuracy: 59.66
CNN ---- Clean Accuracy: 86.03, Noisy Accuracy: 66.46, LDA Clean Accuracy: 88.00, LDA Noisy Accuracy: 63.97
